In [1]:
import pandas as pd
import numpy as np

## Reading data

In [2]:
large = True

if large:
    path = "market_value model\large"
else:
    path = "market_value model"

shooting = pd.read_csv(path + r"\mv-simulation-shooting.csv", index_col=0)
passing = pd.read_csv(path + r"\mv-simulation-passing.csv", index_col=0)
defense = pd.read_csv(path + r"\mv-simulation-defense.csv", index_col=0)
goalkeeping = pd.read_csv(path + r"\mv-simulation-goalkeeping.csv", index_col=0)

player_shooting = pd.read_csv("..\data\processed-data\l_shooting_clean.csv", index_col=0)
player_passing = pd.read_csv("..\data\processed-data\l_passing_clean_90.csv", index_col=0)
player_defense = pd.read_csv("..\data\processed-data\l_defense_clean.csv", index_col=0)
player_goalkeeping = pd.read_csv("..\data\processed-data\l_goalkeeping_clean.csv", index_col=0)

all_players = pd.concat([
    player_shooting[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_passing[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_defense[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_goalkeeping[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    ])
all_players = all_players[all_players.Year == 2021].drop_duplicates().reset_index(drop=True)
print(all_players.shape)
all_players = all_players.assign(Raritan = lambda x: x.Nation == "Rarita")

(2822, 9)


In [3]:
print(all_players[all_players[["Player", "Nation", "Age"]].duplicated()].sort_values(by = "Player").shape)

all_players_group = all_players.groupby(["Player", "Nation", "Age"])
all_players = all_players_group[['FW', 'MF', 'DF', 'GK', 'Raritan']].any().reset_index()
print(all_players.shape)

(105, 10)
(2717, 8)


In [4]:
trials = max(shooting.trial) + 1

# Remove duplicates in players
shooting = shooting.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
passing = passing.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
defense = defense.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
goalkeeping = goalkeeping.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()

In [5]:
pd.options.display.max_rows = 1000
all_players[["FW","MF", "DF", "GK", "Raritan"]] = all_players[["FW","MF", "DF", "GK", "Raritan"]].astype(int)
if large:
    all_players.to_csv(r"..\data\optim-data\large\0_selection_players.csv")
else:
    all_players.to_csv(r"..\data\optim-data\0_selection_players.csv")
all_players

,Player,Nation,Age,FW,MF,DF,GK,Raritan
0,a.aceng,Esia,24,0,0,1,0,0
1,a.acheng,Southern Ristan,24,0,0,1,0,0
2,a.agyapong,Imaar Vircoand,22,0,1,0,0,0
3,a.ainomugisha,Moaithe,20,0,0,1,0,0
4,a.akoth,Unicorporated Tiagascar,27,0,1,1,0,0
...,...,...,...,...,...,...,...,...
2712,z.vilanculos,Esia,23,0,1,1,0,0
2713,z.wati,Dosqaly,22,0,1,0,0,0
2714,z.wu,Ledian,25,0,0,1,0,0
2715,z.yosef,People's Land of Maneau,20,0,1,0,0,0


In [6]:
all_players['player_id'] = all_players.apply(lambda row: "-".join([row[0], row[1]]), axis=1)

shooting['player_id'] = shooting.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
passing['player_id'] = passing.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
defense['player_id'] = defense.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
goalkeeping['player_id'] = goalkeeping.apply(lambda row: "-".join([row[0], row[1]]), axis=1)

shooting.index = pd.MultiIndex.from_frame(shooting[['player_id', 'trial']])
passing.index = pd.MultiIndex.from_frame(passing[['player_id', 'trial']])
defense.index = pd.MultiIndex.from_frame(defense[['player_id', 'trial']])
goalkeeping.index = pd.MultiIndex.from_frame(goalkeeping[['player_id', 'trial']])

In [7]:
from itertools import product
player_mv = pd.DataFrame()
player_mv.index = pd.MultiIndex.from_tuples(list(product(all_players.player_id, np.arange(trials))))

for i in all_players.index:
    if (i % 100 == 0):
        print(f"{i} out of {all_players.shape[0]}")

    player_id = "-".join([all_players.iloc[i, 0], all_players.iloc[i, 1]])
    
    role_FW = all_players.iloc[i, 3]
    role_MF = all_players.iloc[i, 4]
    role_DF = all_players.iloc[i, 5]
    role_GK = all_players.iloc[i, 6]

    best_role = -1
    mv = float("-Inf")
    best_df = None
    for role_i, role in enumerate([role_FW, role_MF, role_DF, role_GK]):
        if role_i == 0:
            df = shooting
        elif role_i == 1:
            df = passing
        elif role_i == 2:
            df = defense
        elif role_i == 3:
            df = goalkeeping
        
        try:
            if best_role == -1 or (best_role != -1 and df.loc[(player_id, 0), "mv0"] > mv):
                mv = df.loc[(player_id, 0), "mv0"]
                best_role = role_i
                best_df = df
        except KeyError:
            pass
    
    all_players.iloc[i, 3] = 0
    all_players.iloc[i, 4] = 0
    all_players.iloc[i, 5] = 0
    all_players.iloc[i, 6] = 0

    try:
        all_players.iloc[i, best_role] = 1
    except:
        pass

    for j in np.arange(trials):
        for t in np.arange(10):
            mv_t = "mv" + str(t)
            
            try:
                player_mv.loc[(player_id, j), mv_t] = best_df.loc[(player_id, j), mv_t]
            except:
                break

player_mv = player_mv.reset_index()

0 out of 2717
100 out of 2717
200 out of 2717
300 out of 2717
400 out of 2717
500 out of 2717
600 out of 2717
700 out of 2717
800 out of 2717
900 out of 2717
1000 out of 2717
1100 out of 2717
1200 out of 2717
1300 out of 2717
1400 out of 2717
1500 out of 2717
1600 out of 2717
1700 out of 2717
1800 out of 2717
1900 out of 2717
2000 out of 2717
2100 out of 2717
2200 out of 2717
2300 out of 2717
2400 out of 2717
2500 out of 2717
2600 out of 2717
2700 out of 2717


In [8]:
player_mv = player_mv.fillna(0)
player_mv = player_mv.rename(columns={"level_0": "player_id", "level_1": "trial"})
player_mv = player_mv.sort_values(by = ["trial", "player_id"])
if large:
    player_mv.to_csv(r"..\data\optim-data\large\0_selection_mv.csv")    
else:
    player_mv.to_csv(r"..\data\optim-data\0_selection_mv.csv")

In [9]:
player_mv

,player_id,trial,mv0,mv1,mv2,mv3,mv4,mv5,mv6,mv7,mv8,mv9
0,a.aceng-Esia,0,13700000.0,12700000.0,11500000.0,10300000.0,9100000.0,7800000.0,6600000.0,5500000.0,4500000.0,3600000.0
2000,a.acheng-Southern Ristan,0,39200000.0,36200000.0,33200000.0,30100000.0,27000000.0,24000000.0,21100000.0,18300000.0,15800000.0,13500000.0
4000,a.agyapong-Imaar Vircoand,0,9500000.0,9100000.0,8600000.0,8000000.0,7200000.0,6500000.0,5700000.0,4900000.0,4200000.0,3500000.0
6000,a.ainomugisha-Moaithe,0,35200000.0,35000000.0,34300000.0,32900000.0,31100000.0,28800000.0,26200000.0,23400000.0,20600000.0,17800000.0
8000,a.akoth-Unicorporated Tiagascar,0,12450000.0,10950000.0,9450000.0,8050000.0,6700000.0,5450000.0,4350000.0,3450000.0,2700000.0,2050000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5425999,z.vilanculos-Esia,1999,8800000.0,4800000.0,11300000.0,5800000.0,18400000.0,1000000.0,4300000.0,7900000.0,2600000.0,100000.0
5427999,z.wati-Dosqaly,1999,7100000.0,7200000.0,23600000.0,17400000.0,7500000.0,10300000.0,7000000.0,1400000.0,10300000.0,4200000.0
5429999,z.wu-Ledian,1999,168900000.0,12900000.0,72200000.0,4700000.0,35800000.0,28200000.0,3300000.0,16200000.0,3700000.0,5000000.0
5431999,z.yosef-People's Land of Maneau,1999,2500000.0,1600000.0,4600000.0,7300000.0,30400000.0,12000000.0,14100000.0,15900000.0,8800000.0,7400000.0
